In [11]:
import boto3
import xmltodict
import json
import pickle
import pandas as pd
import datetime

In [2]:
create_hits_in_production = True
environments = {
  "production": {
    "endpoint": "https://mturk-requester.us-east-1.amazonaws.com",
    "preview": "https://www.mturk.com/mturk/preview"
  },
  "sandbox": {
    "endpoint": 
          "https://mturk-requester-sandbox.us-east-1.amazonaws.com",
    "preview": "https://workersandbox.mturk.com/mturk/preview"
  },
}
mturk_environment = environments["production"] if create_hits_in_production else environments["sandbox"]
session = boto3.Session(profile_name='mturk')
client = session.client(
    service_name='mturk',
    region_name='us-east-1',
    endpoint_url=mturk_environment['endpoint'],
)

In [3]:
print(client.get_account_balance()['AvailableBalance'])


154.20


# Testing

In [13]:
hits = client.list_hits(MaxResults=100)

In [15]:
len(hits['HITs'])

90

In [16]:
hits['HITs'][0].keys()

dict_keys(['HITId', 'HITTypeId', 'HITGroupId', 'HITLayoutId', 'CreationTime', 'Title', 'Description', 'Question', 'Keywords', 'HITStatus', 'MaxAssignments', 'Reward', 'AutoApprovalDelayInSeconds', 'Expiration', 'AssignmentDurationInSeconds', 'RequesterAnnotation', 'QualificationRequirements', 'HITReviewStatus', 'NumberOfAssignmentsPending', 'NumberOfAssignmentsAvailable', 'NumberOfAssignmentsCompleted'])

In [18]:
hits['HITs'][0]['HITId']

'3YCT0L9OMMJ03QGXPN8IR4I5IX0SN8'

In [21]:
assignmentsList = client.list_assignments_for_hit(
    HITId='3YCT0L9OMMJ03QGXPN8IR4I5IX0SN8',
    AssignmentStatuses=['Submitted', 'Approved'],
    MaxResults=10
)

In [48]:
assignmentsList

{'NextToken': 'p2:DnUpxQAfdQxDx7adgHyyWo2iC7gkFMJa9RDLFvWtdl50BjBBDj/JeJYqpxUX7w==',
 'NumResults': 9,
 'Assignments': [{'AssignmentId': '3LOTDFNYA89K9ZIJ16484885K8ZFWO',
   'WorkerId': 'A345TDMHP3DQ3G',
   'HITId': '3YCT0L9OMMJ03QGXPN8IR4I5IX0SN8',
   'AssignmentStatus': 'Approved',
   'AutoApprovalTime': datetime.datetime(2021, 11, 23, 11, 59, 2, tzinfo=tzlocal()),
   'AcceptTime': datetime.datetime(2021, 11, 20, 11, 44, 35, tzinfo=tzlocal()),
   'SubmitTime': datetime.datetime(2021, 11, 20, 11, 59, 2, tzinfo=tzlocal()),
   'ApprovalTime': datetime.datetime(2021, 11, 22, 16, 46, 52, tzinfo=tzlocal()),
   'Answer': '<?xml version="1.0" encoding="ASCII"?><QuestionFormAnswers xmlns="http://mechanicalturk.amazonaws.com/AWSMechanicalTurkDataSchemas/2005-10-01/QuestionFormAnswers.xsd"><Answer><QuestionIdentifier>answer1-1</QuestionIdentifier><FreeText>The girl was not down-to-earth at all.</FreeText></Answer><Answer><QuestionIdentifier>answer1-2</QuestionIdentifier><FreeText>The girl was v

In [37]:
answers = xmltodict.parse(assignmentsList['Assignments'][0]['Answer'])['QuestionFormAnswers']['Answer']
d = {}
for item in answers:
    d[item['QuestionIdentifier']] = item['FreeText']

In [38]:
d

{'answer1-1': 'The girl was not down-to-earth at all.',
 'answer1-2': 'The girl was very fickle.',
 'answer1-3': "The girl's room was a total mess.",
 'answer2-1': 'The girl was very down-to-earth.',
 'answer2-2': 'The girl was very stable.',
 'answer2-3': "The girl's room was very clean.",
 'metaphor-base-1': 'The girl was as down-to-earth as',
 'metaphor-base-2': 'The girl had the flightiness of',
 'metaphor-base-3': "The girl's room was as messy as",
 'phrase1-1': 'a Michelin-starred canape',
 'phrase1-2': 'a sparrow',
 'phrase1-3': 'pig slops',
 'phrase2-1': 'eggs and potatoes',
 'phrase2-2': 'a rock',
 'phrase2-3': 'a housekeeper'}

In [43]:
csv_rows = []
assignment_id = 'asdf'
for i in (1, 2, 3):
    startphrase1 = d[f'metaphor-base-{i}'] + ' ' + d[f'phrase1-{i}']
    startphrase2 = d[f'metaphor-base-{i}'] + ' ' + d[f'phrase2-{i}']
    ending1 = d[f'answer1-{i}']
    ending2 = d[f'answer2-{i}']
    csv_rows.append([assignment_id, startphrase1, ending1, ending2, 0])
    csv_rows.append([assignment_id, startphrase2, ending1, ending2, 1])
csv_rows

[['asdf',
  'The girl was as down-to-earth as a Michelin-starred canape',
  'The girl was not down-to-earth at all.',
  'The girl was very down-to-earth.',
  0],
 ['asdf',
  'The girl was as down-to-earth as eggs and potatoes',
  'The girl was not down-to-earth at all.',
  'The girl was very down-to-earth.',
  1],
 ['asdf',
  'The girl had the flightiness of a sparrow',
  'The girl was very fickle.',
  'The girl was very stable.',
  0],
 ['asdf',
  'The girl had the flightiness of a rock',
  'The girl was very fickle.',
  'The girl was very stable.',
  1],
 ['asdf',
  "The girl's room was as messy as pig slops",
  "The girl's room was a total mess.",
  "The girl's room was very clean.",
  0],
 ['asdf',
  "The girl's room was as messy as a housekeeper",
  "The girl's room was a total mess.",
  "The girl's room was very clean.",
  1]]

# Let's go

In [56]:

csv_rows = [['HITId', 'AssignmentId', 'startphrase', 'ending1', 'ending2', 'label']]

batch_size = 9
hits = client.list_hits(MaxResults=100)
print(f"Found {len(hits['HITs'])} HITs")
for hit_i, HIT in enumerate(hits['HITs']):
    hit_id = HIT['HITId']
    assignmentsList = client.list_assignments_for_hit(
        HITId=hit_id,
        AssignmentStatuses=['Submitted', 'Approved'],
        MaxResults=batch_size
    )
    num_results = assignmentsList['NumResults']
    print(f"{hit_i} found {num_results} results for hit_id {hit_id}")
    for assignment_i in range(num_results):
        asm = assignmentsList['Assignments'][assignment_i]
        assignment_id = asm['AssignmentId']
        answers_xml = xmltodict.parse(asm['Answer'])
        d = {}
        for item in answers_xml['QuestionFormAnswers']['Answer']:
            d[item['QuestionIdentifier']] = item['FreeText']
            
        for i in (1, 2, 3):
            try:
                startphrase1 = d[f'metaphor-base-{i}'] + ' ' + d[f'phrase1-{i}']
                startphrase2 = d[f'metaphor-base-{i}'] + ' ' + d[f'phrase2-{i}']
                ending1 = d[f'answer1-{i}']
                ending2 = d[f'answer2-{i}']
                csv_rows.append([hit_id, assignment_id, startphrase1, ending1, ending2, 0])
                csv_rows.append([hit_id, assignment_id, startphrase2, ending1, ending2, 1])
            except KeyError:
                print(f"there is an error with {i}")

Found 90 HITs
0 found 9 results for hit_id 3YCT0L9OMMJ03QGXPN8IR4I5IX0SN8
1 found 9 results for hit_id 3B623HUYJ40F0JTTJLJZ97SAT21S8K
2 found 9 results for hit_id 3DQYSJDTYLL451PJQYBM0P5OJK1XEW
3 found 9 results for hit_id 324N5FAHSXLFNCCYM5L7AZL8N6PVKB
4 found 9 results for hit_id 34R0BODSP19V3I59KSSR86E4QLS5EV
5 found 9 results for hit_id 3IJ95K7NDXMXDL76UNE12QZ2UM6NGJ
6 found 9 results for hit_id 3UOMW19E6DGG6KLVO1MWIIE159F5C2
7 found 9 results for hit_id 356TQKY9XF7THL91VILOEGWE6OP78D
8 found 9 results for hit_id 33N1S8XHHMVUQVVMFS4SQQ252BW1ZF
9 found 8 results for hit_id 338431Z1FLP6KWGWBD8QGJDEM5MROY
10 found 9 results for hit_id 3K2CEDRACBBV088C2MY8OUPRFQ4MTR
11 found 8 results for hit_id 32LAQ1JNT9ZEDGDLBOPUF46LSC7UTR
12 found 9 results for hit_id 3J6BHNX0U922F73PKEZG0IRV13NKNJ
13 found 9 results for hit_id 39I4RL8QGJRPR9SBWOHNOH53I5P4H5
14 found 9 results for hit_id 3MD8CKRQZZXSJLPZV0GL8NWSNBCRJF
15 found 9 results for hit_id 3IZPORCT1FJWSFU970D0WAGDAJ8RHA
16 found 9 results f

In [57]:
len(csv_rows)

4645

In [58]:
csv_rows[123]

['3DQYSJDTYLL451PJQYBM0P5OJK1XEW',
 '3OVR4I9USQT5RMP85G9JLFTS7IQQ4Q',
 'She sings like an angel',
 'her voice is magical',
 'her voice is awful',
 0]

In [60]:
import pandas as pd
pd.DataFrame(csv_rows).to_csv('mturk_answers_full.csv', header=False, index=False)

# Launching some batches

In [61]:
HIT_type_id = "3L55M9M85Z2K6NTTAODL9E6BAIK9OT"
HIT_layout_id = "3VFQ2DTUL3YVR9W7SFAN2H76RYT8GT"

In [91]:
# w1, w2, w3 = 'cheat', 'night', 'digest'
# 121
def launch_batch(w1, w2, w3):
    response = client.create_hit_with_hit_type(
        HITTypeId=HIT_type_id,
        MaxAssignments=9,
        LifetimeInSeconds=604800,  # 7 days
        HITLayoutId=HIT_layout_id,
        HITLayoutParameters=[
        {
            'Name': 'w1',
            'Value': w1
        },
        {
            'Name': 'w2',
            'Value': w2
        },
        {
            'Name': 'w3',
            'Value': w3
        },
    ]
    )
    return response

In [99]:
# responses = []
responses = [response]  # previous one. delete this line if running notebook fresh

for i in range(150, 301):
    print(i)
    with open(f"batch_csvs/batch_{i}.csv") as f:
        w1, w2, w3 = f.read().split('\n')[1].split(',')
    responses.append(launch_batch(w1, w2, w3))


150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300


In [100]:
import pickle
with open("batch_csvs/batch_create_responses.pkl", 'wb') as f:
    pickle.dump(responses, f)

In [98]:
with open("batch_csvs/test.pkl", 'rb') as f:
    res = pickle.load(f)
res

{'HIT': {'HITId': '389A2A304OSHBDZURVGK1NIAO700CK',
  'HITTypeId': '3L55M9M85Z2K6NTTAODL9E6BAIK9OT',
  'HITGroupId': '3SU8TQ3QV4O8K6Z09HLV7Y3QZ2VIE4',
  'HITLayoutId': '3VFQ2DTUL3YVR9W7SFAN2H76RYT8GT',
  'CreationTime': datetime.datetime(2021, 11, 25, 16, 26, 23, tzinfo=tzlocal()),
  'Title': 'Create a creative metaphor using the template',
  'Description': 'Filling in the template below (select a subject, fill in a metaphorical phrase, and write the interpretation in the template)',
  'Question': '<?xml version="1.0"?>\n<HTMLQuestion xmlns="http://mechanicalturk.amazonaws.com/AWSMechanicalTurkDataSchemas/2011-11-11/HTMLQuestion.xsd">\n  <HTMLContent><![CDATA[<html><head><title>HIT</title><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"/></head><body><!-- You must include this JavaScript file -->\r\n<script src="https://assets.crowd.aws/crowd-html-elements.js"></script>\r\n\r\n<script>\r\nfunction setPrefix(index) {\r\n  document.getElementById(\'prefix1-\' + index).i

In [80]:
hits = client.list_hits(MaxResults=100)


In [81]:
hits['HITs'][0]

{'HITId': '389A2A304OSHBDZURVGK1NIAO700CK',
 'HITTypeId': '3L55M9M85Z2K6NTTAODL9E6BAIK9OT',
 'HITGroupId': '3SU8TQ3QV4O8K6Z09HLV7Y3QZ2VIE4',
 'HITLayoutId': '3VFQ2DTUL3YVR9W7SFAN2H76RYT8GT',
 'CreationTime': datetime.datetime(2021, 11, 25, 16, 26, 23, tzinfo=tzlocal()),
 'Title': 'Create a creative metaphor using the template',
 'Description': 'Filling in the template below (select a subject, fill in a metaphorical phrase, and write the interpretation in the template)',
 'Question': '<?xml version="1.0"?>\n<HTMLQuestion xmlns="http://mechanicalturk.amazonaws.com/AWSMechanicalTurkDataSchemas/2011-11-11/HTMLQuestion.xsd">\n  <HTMLContent><![CDATA[<html><head><title>HIT</title><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"/></head><body><!-- You must include this JavaScript file -->\r\n<script src="https://assets.crowd.aws/crowd-html-elements.js"></script>\r\n\r\n<script>\r\nfunction setPrefix(index) {\r\n  document.getElementById(\'prefix1-\' + index).innerHTML = docu

In [84]:
my_ghost_HITId = "389A2A304OSHBDZURVGK1NIAO700CK"
client.get_hit(HITId=my_ghost_HITId)

{'HIT': {'HITId': '389A2A304OSHBDZURVGK1NIAO700CK',
  'HITTypeId': '3L55M9M85Z2K6NTTAODL9E6BAIK9OT',
  'HITGroupId': '3SU8TQ3QV4O8K6Z09HLV7Y3QZ2VIE4',
  'HITLayoutId': '3VFQ2DTUL3YVR9W7SFAN2H76RYT8GT',
  'CreationTime': datetime.datetime(2021, 11, 25, 16, 26, 23, tzinfo=tzlocal()),
  'Title': 'Create a creative metaphor using the template',
  'Description': 'Filling in the template below (select a subject, fill in a metaphorical phrase, and write the interpretation in the template)',
  'Question': '<?xml version="1.0"?>\n<HTMLQuestion xmlns="http://mechanicalturk.amazonaws.com/AWSMechanicalTurkDataSchemas/2011-11-11/HTMLQuestion.xsd">\n  <HTMLContent><![CDATA[<html><head><title>HIT</title><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"/></head><body><!-- You must include this JavaScript file -->\r\n<script src="https://assets.crowd.aws/crowd-html-elements.js"></script>\r\n\r\n<script>\r\nfunction setPrefix(index) {\r\n  document.getElementById(\'prefix1-\' + index).i

# Check progress of launched batches

In [6]:
with open("batch_csvs/batch_create_responses.pkl", 'rb') as f:
    turk_round2 = pickle.load(f)
turk_round2

[{'HIT': {'HITId': '389A2A304OSHBDZURVGK1NIAO700CK',
   'HITTypeId': '3L55M9M85Z2K6NTTAODL9E6BAIK9OT',
   'HITGroupId': '3SU8TQ3QV4O8K6Z09HLV7Y3QZ2VIE4',
   'HITLayoutId': '3VFQ2DTUL3YVR9W7SFAN2H76RYT8GT',
   'CreationTime': datetime.datetime(2021, 11, 25, 16, 26, 23, tzinfo=tzlocal()),
   'Title': 'Create a creative metaphor using the template',
   'Description': 'Filling in the template below (select a subject, fill in a metaphorical phrase, and write the interpretation in the template)',
   'Question': '<?xml version="1.0"?>\n<HTMLQuestion xmlns="http://mechanicalturk.amazonaws.com/AWSMechanicalTurkDataSchemas/2011-11-11/HTMLQuestion.xsd">\n  <HTMLContent><![CDATA[<html><head><title>HIT</title><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"/></head><body><!-- You must include this JavaScript file -->\r\n<script src="https://assets.crowd.aws/crowd-html-elements.js"></script>\r\n\r\n<script>\r\nfunction setPrefix(index) {\r\n  document.getElementById(\'prefix1-\' + 

In [7]:
len(turk_round2)

152

In [10]:
turk_round2[0]['ResponseMetadata']

{'RequestId': '8d5c2aad-4599-4582-9be0-41655166c589',
 'HTTPStatusCode': 200,
 'HTTPHeaders': {'x-amzn-requestid': '8d5c2aad-4599-4582-9be0-41655166c589',
  'content-type': 'application/x-amz-json-1.1',
  'content-length': '13237',
  'date': 'Thu, 25 Nov 2021 21:26:23 GMT'},
 'RetryAttempts': 0}

In [14]:
now = datetime.datetime.now().strftime('%Y-%m-%d-%H:%M:%S')
csv_rows = [['HITId', 'num_completed', 'progress']]

batch_size = 9

for hit_i, HIT in enumerate(turk_round2):
    hit_id = HIT['HIT']['HITId']
    assignmentsList = client.list_assignments_for_hit(
        HITId=hit_id,
        AssignmentStatuses=['Submitted', 'Approved'],
        MaxResults=batch_size
    )
    num_results = assignmentsList['NumResults']
    print(f"{hit_i} found {num_results} results for hit_id {hit_id}")
    progress_bar = "|"+'█'*num_results + ' '*(9-num_results) + "|"
    csv_rows.append([hit_id, num_results, progress_bar])
#     for assignment_i in range(num_results):
#         asm = assignmentsList['Assignments'][assignment_i]
#         assignment_id = asm['AssignmentId']
#         answers_xml = xmltodict.parse(asm['Answer'])
#         d = {}
#         for item in answers_xml['QuestionFormAnswers']['Answer']:
#             d[item['QuestionIdentifier']] = item['FreeText']
            
#         for i in (1, 2, 3):
#             try:
#                 startphrase1 = d[f'metaphor-base-{i}'] + ' ' + d[f'phrase1-{i}']
#                 startphrase2 = d[f'metaphor-base-{i}'] + ' ' + d[f'phrase2-{i}']
#                 ending1 = d[f'answer1-{i}']
#                 ending2 = d[f'answer2-{i}']
#                 csv_rows.append([hit_id, assignment_id, startphrase1, ending1, ending2, 0])
#                 csv_rows.append([hit_id, assignment_id, startphrase2, ending1, ending2, 1])
#             except KeyError:
#                 print(f"there is an error with {i}")

0 found 9 results for hit_id 389A2A304OSHBDZURVGK1NIAO700CK
1 found 9 results for hit_id 3G9UA71JVV4I125KMMIF365SM7HJ7N
2 found 9 results for hit_id 3R5OYNIC2CJ1LB5PU2Q05YJMK3ETP9
3 found 9 results for hit_id 3CO05SML7VFLHFOD9S6FQIHWOA5R0N
4 found 9 results for hit_id 37G6BXQPLQVKW4O74KGU6H9VMPDEQV
5 found 9 results for hit_id 3IV1AEQ4DRNM1GA3WRALH61NXRNJ8A
6 found 9 results for hit_id 3XU9MCX6VOMSDPRROFAT96ABL8PR27
7 found 9 results for hit_id 3R6RZGK0XFMA7H1FQKMYJ70JLTXVYL
8 found 9 results for hit_id 3VAOOVPI3Z2NBDIMT1V22Q0LLGRLL0
9 found 9 results for hit_id 3HJ1EVZS2OTHY7NASFOYSGF3RMPR3I
10 found 9 results for hit_id 3QO7EE372OXMP1CNUTE4593AJS0BQS
11 found 9 results for hit_id 3L60IFZKF3SJ3LG6BV16I6E7NI9HHI
12 found 9 results for hit_id 3EKTG13IZUD7HV07TDWH23FALLHLM2
13 found 9 results for hit_id 39RRBHZ0AUBAC7BKM6OHXE9LRN1VZC
14 found 9 results for hit_id 34O39PNDK6IM12RN7834AZDFFE2BRT
15 found 9 results for hit_id 36BTXXLZ2VI2G36ESTTH2L795MLR43
16 found 9 results for hit_id 3421

134 found 9 results for hit_id 3TZDZ3Y0JSGFK4D9BUATD20CNH619G
135 found 9 results for hit_id 3WRKFXQBOBH5LBHFRDNF4IAPDR1YIR
136 found 9 results for hit_id 3ODOP6T3ASULCJPTD3EG1DZ1TV6424
137 found 9 results for hit_id 38LRF35D5L69EZFRW9J3XR2NYLTU3P
138 found 9 results for hit_id 351S7I5UG9674NX72TZ4Z3RIMMMJNY
139 found 9 results for hit_id 378G7J1SJL8PG1KTGTF0Y6RHAQUWEH
140 found 9 results for hit_id 3W5PY7V3UPH1GM1L9V8FGJPPQ6QYJ3
141 found 9 results for hit_id 382GHPVPHS1AX1LCH3SLKN4TZ9643F
142 found 9 results for hit_id 3XDJY5RK5SVUWVYVWNOM7WUTCLPU4A
143 found 9 results for hit_id 3ATYLI1PRTCPM8XJ38ZHESRNCWRJOD
144 found 9 results for hit_id 3AA88CN98PDWR673RG8RHYP25VQYK2
145 found 9 results for hit_id 3MD8CKRQZZXSJLPZV0GL8NWSNJBRJU
146 found 9 results for hit_id 3P520RYKCHGCSE2VAS7WCO07CHKU5F
147 found 9 results for hit_id 3XU80RHWHZQVFX4GHHX4USWZD92440
148 found 9 results for hit_id 3KTCJ4SCVGBUROW303EYIO6WNFCM1X
149 found 9 results for hit_id 3FTID4TN8L87BCPLCGKSWY2HUVVYL5
150 foun

In [15]:
pd.DataFrame(csv_rows).to_csv(f"mturk_progress_{now}.csv")

# Extract HITs, round 2

In [25]:

csv_rows = [['HITId', 'AssignmentId', 'startphrase', 'ending1', 'ending2', 'label']]

batch_size = 9
for hit_i, HIT in enumerate(turk_round2):
    hit_id = HIT['HIT']['HITId']
    assignmentsList = client.list_assignments_for_hit(
        HITId=hit_id,
        AssignmentStatuses=['Submitted', 'Approved'],
        MaxResults=batch_size
    )
    num_results = assignmentsList['NumResults']
    print(f"{hit_i} found {num_results} results for hit_id {hit_id}")
    for assignment_i in range(num_results):
        asm = assignmentsList['Assignments'][assignment_i]
        assignment_id = asm['AssignmentId']
        answers_xml = xmltodict.parse(asm['Answer'])
        d = {}
        for item in answers_xml['QuestionFormAnswers']['Answer']:
            d[item['QuestionIdentifier']] = item['FreeText']
            
        for i in (1, 2, 3):
            try:
                startphrase1 = d[f'metaphor-base-{i}'] + ' ' + d[f'phrase1-{i}']
                startphrase2 = d[f'metaphor-base-{i}'] + ' ' + d[f'phrase2-{i}']
                ending1 = d[f'answer1-{i}']
                ending2 = d[f'answer2-{i}']
                csv_rows.append([hit_id, assignment_id, startphrase1, ending1, ending2, 0])
                csv_rows.append([hit_id, assignment_id, startphrase2, ending1, ending2, 1])
            except KeyError:
                print(f"there is an error with {i}")

0 found 9 results for hit_id 389A2A304OSHBDZURVGK1NIAO700CK
1 found 9 results for hit_id 3G9UA71JVV4I125KMMIF365SM7HJ7N
2 found 9 results for hit_id 3R5OYNIC2CJ1LB5PU2Q05YJMK3ETP9
3 found 9 results for hit_id 3CO05SML7VFLHFOD9S6FQIHWOA5R0N
4 found 9 results for hit_id 37G6BXQPLQVKW4O74KGU6H9VMPDEQV
5 found 9 results for hit_id 3IV1AEQ4DRNM1GA3WRALH61NXRNJ8A
6 found 9 results for hit_id 3XU9MCX6VOMSDPRROFAT96ABL8PR27
7 found 9 results for hit_id 3R6RZGK0XFMA7H1FQKMYJ70JLTXVYL
8 found 9 results for hit_id 3VAOOVPI3Z2NBDIMT1V22Q0LLGRLL0
9 found 9 results for hit_id 3HJ1EVZS2OTHY7NASFOYSGF3RMPR3I
10 found 9 results for hit_id 3QO7EE372OXMP1CNUTE4593AJS0BQS
11 found 9 results for hit_id 3L60IFZKF3SJ3LG6BV16I6E7NI9HHI
12 found 9 results for hit_id 3EKTG13IZUD7HV07TDWH23FALLHLM2
13 found 9 results for hit_id 39RRBHZ0AUBAC7BKM6OHXE9LRN1VZC
14 found 9 results for hit_id 34O39PNDK6IM12RN7834AZDFFE2BRT
15 found 9 results for hit_id 36BTXXLZ2VI2G36ESTTH2L795MLR43
16 found 9 results for hit_id 3421

135 found 9 results for hit_id 3WRKFXQBOBH5LBHFRDNF4IAPDR1YIR
136 found 9 results for hit_id 3ODOP6T3ASULCJPTD3EG1DZ1TV6424
137 found 9 results for hit_id 38LRF35D5L69EZFRW9J3XR2NYLTU3P
138 found 9 results for hit_id 351S7I5UG9674NX72TZ4Z3RIMMMJNY
139 found 9 results for hit_id 378G7J1SJL8PG1KTGTF0Y6RHAQUWEH
140 found 9 results for hit_id 3W5PY7V3UPH1GM1L9V8FGJPPQ6QYJ3
141 found 9 results for hit_id 382GHPVPHS1AX1LCH3SLKN4TZ9643F
142 found 9 results for hit_id 3XDJY5RK5SVUWVYVWNOM7WUTCLPU4A
143 found 9 results for hit_id 3ATYLI1PRTCPM8XJ38ZHESRNCWRJOD
144 found 9 results for hit_id 3AA88CN98PDWR673RG8RHYP25VQYK2
145 found 9 results for hit_id 3MD8CKRQZZXSJLPZV0GL8NWSNJBRJU
146 found 9 results for hit_id 3P520RYKCHGCSE2VAS7WCO07CHKU5F
147 found 9 results for hit_id 3XU80RHWHZQVFX4GHHX4USWZD92440
148 found 9 results for hit_id 3KTCJ4SCVGBUROW303EYIO6WNFCM1X
149 found 9 results for hit_id 3FTID4TN8L87BCPLCGKSWY2HUVVYL5
150 found 8 results for hit_id 30ZKOOGW2WGTPNNE30OT9XBWTZO1AV
151 foun

In [26]:
pd.DataFrame(csv_rows).to_csv('mturk_answers_round2.csv', header=False, index=False)